In [ ]:
# Import findspark and initialize. 
import findspark
findspark.init()

ValueError: Couldn't find Spark, make sure SPARK_HOME env is set or Spark is in an expected location (e.g. from homebrew installation).

In [ ]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [ ]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"


#creating the dataframe 

spark.sparkContext.addFile(url)
home_df = spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True)


In [ ]:
# 2. Create a temporary view of the DataFrame.

home_df.show()  #i wanted to see how the dataframe looked like

home_df.createOrReplaceTempView('home_sales')  #the bootcamp spot document advised to use "home_sales"


In [ ]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

avg_price_4b = """
(SELECT date_built, ROUND(avg(price),2)
FROM home_sales
WHERE bedrooms==4 
GROUP BY date_built
ORDER BY date_built ASC)
"""

spark.sql(avg_price_4b).show()


In [ ]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?
avg_price_3b_3b = """
(SELECT date_built, ROUND(avg(price),2)
FROM home_sales
WHERE bedrooms==3 AND bathrooms==3
GROUP BY date_built
ORDER BY date_built ASC)
"""

spark.sql(avg_price_3b_3b).show()


In [ ]:
#  5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?


avg_price_3b_3b_2f = """
(SELECT date_built, ROUND(avg(price),2)
FROM home_sales
WHERE bedrooms==3 AND bathrooms==3 AND floors==2 AND sqft_living>=2000
GROUP BY date_built
ORDER BY date_built ASC)
"""

spark.sql(avg_price_3b_3b_2f).show()


In [ ]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()


avg_home350k = """
(SELECT view, ROUND(avg(price),2) as average_home_price
FROM home_sales
WHERE price>=350000
GROUP BY view
ORDER BY view ASC)
"""

spark.sql(avg_home350k).show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

In [ ]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

In [ ]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time = time.time()

avg_home350k_cache = """
(SELECT view, ROUND(avg(price),2) as average_home_price
FROM home_sales
WHERE price>=350000
GROUP BY view
ORDER BY view ASC)
"""

spark.sql(avg_home350k_cache).show()

print("--- %s seconds ---" % (time.time() - start_time))


In [ ]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_df.write.parquet('parquet_homes', mode='overwrite')

In [ ]:
# 11. Read the formatted parquet data.
parquet_data=spark.read.parquet('parquet_homes')
#parquet_data   # wanted to view the data

In [ ]:
# 12. Create a temporary table for the parquet data.
parquet_data.createOrReplaceTempView('parquet_homes_temp')   #i used parquet_data instead of home_sales because it made more sense

In [ ]:
# 13. Run the query that filters out the view ratings with average price of greater than or eqaul to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version. 

start_time = time.time()


start_time = time.time()

spark.sql("""
(SELECT view, ROUND(avg(price),2)
FROM parquet_homes_temp
WHERE price>=350000
GROUP BY view
ORDER BY view ASC)
""").show()


print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

In [ ]:
# 15. Check if the home_sales is no longer cached

spark.catalog.isCached("home_sales")